In [ ]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor, log
import cv2

import tensorflow as tf
import keras
import cupy as cp
import sys

# from keras_applications.resnet import ResNet50
from keras.applications import *
from keras.layers import *
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
from tensorflow import keras
from PIL import Image
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications import DenseNet121, ResNet50V2, InceptionV3
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.callbacks import TensorBoard
#from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
#from sklearn.model_selection import train_test_split
#from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
if 'checkpoint' not in os.listdir('./'):
    os.mkdir('./checkpoint')

input_path = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/"
test_images_dir = input_path + 'stage_2_test/'
train_images_dir = input_path + 'stage_2_train/'

In [ ]:
path = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/ID_000012eaf.dcm'
img = pydicom.dcmread(path)
img

In [ ]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):
    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img

# Sanity Check
# Example dicoms: ID_2669954a7, ID_5c8b5d701, ID_52c9913b1

dicom = pydicom.dcmread(train_images_dir + 'ID_5c8b5d701' + '.dcm')
 
# 4045571               ID_5c8b5d701_any      1
plt.imshow(bsb_window(dicom), cmap=plt.cm.bone);


In [ ]:
def window_with_correction(dcm, window_center, window_width):
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def window_without_correction(dcm, window_center, window_width):
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def window_testing(img, window):
    brain_img = window(img, 40, 80)
    subdural_img = window(img, 80, 200)
    soft_img = window(img, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img

# example of a "bad data point" (i.e. (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100) == True)
dicom = pydicom.dcmread(train_images_dir + "ID_036db39b7" + ".dcm")

fig, ax = plt.subplots(1, 2)

ax[0].imshow(window_testing(dicom, window_without_correction), cmap=plt.cm.bone);
ax[0].set_title("original")
ax[1].imshow(window_testing(dicom, window_with_correction), cmap=plt.cm.bone);
ax[1].set_title("corrected");

In [ ]:
df = pd.read_csv('../input/rsna-csv-files/RSNA_DATA/DATA.csv')

In [ ]:
from tqdm import trange
def get_partition_labels(df):
    partition = dict()
    labels = dict()
    for i in trange(len(df)):
        id_ = df.Image[i]
        label = df.iloc[i,1:7].to_numpy(dtype = 'int32')
        labels[id_] = label
        
    df = df.sample(frac = 0.1)
    training = df.sample(frac = 0.7)
    
    validation = df.drop(training.index, axis = 0)
    test = validation.sample(frac = 0.5)
    validation = validation.drop(test.index, axis = 0) 
    
    partition['train'] = list(training.Image)
    partition['validation'] = list(validation.Image)
    partition['test'] = list(test.Image)
    return partition,labels

In [ ]:
partition,labels = get_partition_labels(df)

In [ ]:
print(len(partition['train']))
values_view = labels.values()
value_iterator = iter(values_view)
first_value = next(value_iterator)
print(next(iter(labels)))
print(first_value)


values_view2 = partition.values()
value_iterator2 = iter(values_view)
first_value2 = next(value_iterator)
print(next(iter(partition['train'])))
print(first_value2)

In [ ]:
def _read(path, desired_size):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)
    
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(desired_size)
    
    
    img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
    return img

# Another sanity check 
plt.imshow(
    _read(train_images_dir+'ID_5c8b5d701'+'.dcm', (256, 256,3)), cmap=plt.cm.bone
);

In [ ]:
type(_read(train_images_dir+'ID_5c8b5d701'+'.dcm', (256, 256,3)))

In [ ]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=64, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.true_labels = []
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        
        self.true_labels.append(y)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim)) #,self.n_channels))
        
        y = np.empty((self.batch_size,6), dtype=int)
        #print(y.shape)
        #print(X.shape)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            image_dir = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'
            X[i,] = _read(image_dir+ID+'.dcm',self.dim)
            #print(X)
            # Store class
            
            y[i] = self.labels[ID]
            #print(y[i])

        return X,y

In [ ]:
params = {'dim':(224,224,3),
         'batch_size':64,
         'n_classes':6,
         'n_channels':0,
         'shuffle':True}

my_callbacks = [
    ModelCheckpoint(filepath='./checkpoint', monitor = 'val_weighted_loss' ,save_best_only=True,verbose = 3),
    ReduceLROnPlateau(monitor= 'val_weighted_loss', factor=0.1, patience= 3, verbose=1,mode='auto', min_delta=0.0001)]

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = Sequential([
    tf.keras.applications.ResNet152V2(
    include_top=False,
    weights="imagenet",
    input_shape=params['dim'],
    pooling='max'),
    Flatten(),
    Dense(30,activation = 'relu'),
    Dense(6,activation = 'sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.BinaryAccuracy()])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['ACC'])

# Train model on dataset
history = model.fit(training_generator,validation_data=validation_generator, workers = 4, use_multiprocessing= True, epochs=20, callbacks=callbacks)

In [ ]:
def plot_history(history, desc = ''):
    fig = plt.figure(figsize = (18 , 6))
    if desc:
        plt.title('{}'.format(desc), fontsize = 16, y = -0.1)

    subplot = (1, 2, 1)
    fig.add_subplot(*subplot)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train loss', 'valid loss'])
    plt.grid(True)
    plt.plot()
    
    subplot = (1, 2, 2)
    fig.add_subplot(*subplot)
    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['train accuracy', 'valid accuracy'])
    plt.grid(True)
    plt.plot()

In [ ]:
plot_history(history)

In [ ]:
from tensorflow.keras import backend as K

def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights =  tf.Variable([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    out = -(         y_true  * K.log(      y_pred) * class_weights
            + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    
    return K.mean(out, axis=-1)
####
def weighted_log_loss_V2(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights =  tf.constant([2., 1., 1., 1., 1., 1.])
    
    eps = tf.keras.backend.epsilon()
    
    
    y_pred = tf.clip_by_value(y_pred, eps, 1.0-eps)

    out = -(         y_true  * tf.math.log(      y_pred) * class_weights
            + (1.0 - y_true) * tf.math.log(1.0 - y_pred) * class_weights)
    
    return tf.reduce_mean(out, axis=-1)
####

def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for this competition
    """
    
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)


def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    
    class_weights = tf.constant([2., 1., 1., 1., 1., 1.])
    
    eps = tf.keras.backend.epsilon()
    
    y_pred = tf.clip_by_value(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * tf.math.log(      y_pred)
            + (1.0 - y_true) * tf.math.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return tf.reduce_mean(loss_samples)


def weighted_log_loss_metric(trues, preds):
    """
    Will be used to calculate the log loss 
    of the validation set in PredictionCheckpoint()
    ------------------------------------------
    """
    class_weights = [2., 1., 1., 1., 1., 1.]
    
    epsilon = 1e-7
    
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_samples = np.average(loss, axis=1, weights=class_weights)

    return - loss_samples.mean()



In [ ]:
#Evaluation
params_test = {'dim':(224,224,3),
         'batch_size':1,
         'n_classes':6,
         'n_channels':0,
         'shuffle':False}

test_generator = DataGenerator(partition['test'], labels, **params_test)
test_pred = model.predict(test_generator,verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix, classification_report

print(len(partition['test']))
prediction_with_treshold = []

treshold = 0.1

for sample in test_pred:
    prediction_with_treshold.append([1 if i>=treshold else 0 for i in sample ] )
prediction_with_treshold = np.array(prediction_with_treshold)
print(len(prediction_with_treshold))

print(type(test_generator.true_labels[10]))
total_list = np.concatenate(test_generator.true_labels)
total_list = total_list[:-1, :]
print(len(total_list))

accuracy_score(total_list, prediction_with_treshold)
print(multilabel_confusion_matrix(total_list, prediction_with_treshold))
print(classification_report(total_list, prediction_with_treshold, target_names = ['Any', 'Epidural', 'Intraparenychemal','Intraventricular','Subarachnoid','Subdural']))

In [ ]:
from sklearn.metrics import average_precision_score

average_precision_score(total_list, test_pred, average='micro', pos_label=1, sample_weight=None)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix, classification_report

accuracy_score(total_list, prediction_with_treshold)
print(multilabel_confusion_matrix(total_list, prediction_with_treshold))
print(classification_report(total_list, prediction_with_treshold, target_names = ['Any', 'Epidural', 'Intraparenychemal','Intraventricular','Subarachnoid','Subdural']))

In [ ]:
print("test_generator.true_labels")

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
array1 = _read(train_images_dir+"ID_29c9c2ee8"+'.dcm',(224, 224,3))

In [ ]:
last_conv_layer_name = "conv5 x"

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(array1, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
subdural_img = 'ID_29c9c2ee8.dcm'	
intraventricular_img = 'ID_004780f8e.dcm'
intraparenchymal_img = 'ID_000d69988.dcm'
subarachnoid_img = 'ID_00058bb06.dcm'
epidural_img = 'ID_00f1e66e1.dcm'
epidural_img2 = 'ID_ff0afaa64.dcm'
subdural_img2 = 'ID_47f130bfa.dcm'	
subarachnoid_img2 = 'ID_526b45786.dcm'

type_list = [subdural_img, intraventricular_img, intraparenchymal_img, subarachnoid_img, epidural_img2]

In [ ]:
img_id = subarachnoid_img2
img = hem_df.loc[hem_df.index == img_id].index[0]
image = _read(train_images_dir + img,(224, 224))

In [ ]:
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=1)[0])

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
#SUBMITTER UNIT

test_csv = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv"
test_dir = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_test"
TEST_DIR = "stage_2_test/"
test_df = pd.read_csv(test_csv)
test_df.head()

testdf = test_df.ID.str.rsplit("_", n=1, expand=True)
testdf = testdf.rename({0: "id", 1: "subtype"}, axis=1)
testdf.loc[:, "label"] = 0

testdf = pd.pivot_table(testdf, index="id", columns="subtype", values="label")

def preprocess(file,type="WINDOW",DIR=test_images_dir):
    dcm = pydicom.dcmread(test_images_dir+file+".dcm")
    if type == "WINDOW":
        window_center , window_width, intercept, slope = get_windowing(dcm)
        w = window_image(dcm, window_center, window_width)
        win_img = np.repeat(w[:, :, np.newaxis], 3, axis=2)
        #return win_img
    elif type == "SIGMOID":
        window_center , window_width, intercept, slope = get_windowing(dcm)
        test_img = dcm.pixel_array
        w = sigmoid_window(dcm, window_center, window_width)
        win_img = np.repeat(w[:, :, np.newaxis], 3, axis=2)
        #return win_img
    elif type == "BSB":
        win_img = bsb_window(dcm)
        #return win_img
    elif type == "SIGMOID_BSB":
        win_img = sigmoid_bsb_window(dcm)
    elif type == "GRADIENT":
        win_img = rainbow_window(dcm)
        #return win_img
    else:
        win_img = dcm.pixel_array
    resized = cv2.resize(win_img,(224,224))
    return resized

class DataLoader(Sequence):
    def __init__(self, dataframe,
                 batch_size,
                 shuffle,
                 input_shape,
                 num_classes=6,
                 steps=None,
                 prep="SIGMOID"):
        
        self.data_ids = dataframe.index.values
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.current_epoch=0
        self.prep = prep
        self.steps=steps
        if self.steps is not None:
            self.steps = np.round(self.steps/3) * 3
            self.undersample()
        
    def undersample(self):
        part = np.int(self.steps/3 * self.batch_size)
        zero_ids = np.random.choice(self.dataframe.loc[self.dataframe["any"] == 0].index.values, size=5000, replace=False)
        hot_ids = np.random.choice(self.dataframe.loc[self.dataframe["any"] == 1].index.values, size=5000, replace=True)
        self.data_ids = list(set(zero_ids).union(hot_ids))
        np.random.shuffle(self.data_ids)
        
    # defines the number of steps per epoch
    def __len__(self):
        if self.steps is None:
            return np.int(np.ceil(len(self.data_ids) / np.float(self.batch_size)))
        else:
            return 3*np.int(self.steps/3) 
    
    # at the end of an epoch: 
    def on_epoch_end(self):
        # if steps is None and shuffle is true:
        if self.steps is None:
            self.data_ids = self.dataframe.index.values
            if self.shuffle:
                np.random.shuffle(self.data_ids)
        else:
            self.undersample()
        self.current_epoch += 1
    
    # should return a batch of images
    def __getitem__(self, item):
        # select the ids of the current batch
        current_ids = self.data_ids[item*self.batch_size:(item+1)*self.batch_size]
        X, y = self.__generate_batch(current_ids)
        return X, y
    
    # collect the preprocessed images and targets of one batch
    def __generate_batch(self, current_ids):
        X = np.empty((self.batch_size, *self.input_shape, 3))
        y = np.empty((self.batch_size, self.num_classes))
        for idx, ident in enumerate(current_ids):
            # Store sample
            #image = self.preprocessor.preprocess(ident) 
            image = preprocess(ident,self.prep)
            X[idx] = image
            # Store class
            y[idx] = self.__get_target(ident)
        return X, y
    
    # extract the targets of one image id:
    def __get_target(self, ident):
        targets = self.dataframe.loc[ident].values
        return targets
    
def turn_pred_to_dataframe(data_df, pred):
    df = pd.DataFrame(pred, columns=data_df.columns, index=data_df.index)
    df = df.stack().reset_index()
    df.loc[:, "ID"] = df.id.str.cat(df.subtype, sep="_")
    df = df.drop(["id", "subtype"], axis=1)
    df = df.rename({0: "Label"}, axis=1)
    return df

def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    
    class_weights = tf.constant([2., 1., 1., 1., 1., 1.])
    
    eps = tf.keras.backend.epsilon()
    
    y_pred = tf.clip_by_value(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * tf.math.log(      y_pred)
            + (1.0 - y_true) * tf.math.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return tf.reduce_mean(loss_samples)

import keras.losses

test_dataloader = DataLoader(testdf,32,shuffle=False,input_shape=(224,224),prep="SIGMOID")

test_pred = model.predict(workers = 2, use_multiprocessing= True, test_dataloader,verbose=1)

pred = test_pred[0:testdf.shape[0]]
pred_df = turn_pred_to_dataframe(testdf,pred)
pred_df.to_csv("test4.csv",index=False)

In [ ]:
def sigmoid_window(dcm, window_center, window_width, U=1.0, eps=(1.0 / 255.0)):
    img = dcm.pixel_array
    img = cp.array(np.array(img))
    _, _, intercept, slope = get_windowing(dcm)
    img = img * slope + intercept
    ue = cp.log((U / eps) - 1.0)
    W = (2 / window_width) * ue
    b = ((-2 * window_center) / window_width) * ue
    z = W * img + b
    img = U / (1 + cp.power(np.e, -1.0 * z))
    img = (img - cp.min(img)) / (cp.max(img) - cp.min(img))
    return cp.asnumpy(img)
def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

In [ ]:
test_generator = DataGenerator(partition['test'], labels, **params)

print("Test Accuracy: ", model.evaluate(test_generator))

In [ ]:
ds=test_generator
predictions= model.predict(ds)
print("A sample output from the last layer (model) ", predictions[0])
y=[]
print("10 Sample predictions:")
for (pred,(a,b)) in zip(predictions,ds):
  
  pred[pred>0.5]=1
  pred[pred<=0.5]=0
  print("predicted: " ,pred, str(covert_onehot_string_labels(LABELS, pred)),  
        "Actual Label: ("+str(covert_onehot_string_labels(LABELS,b.numpy())) +")")
  y.append(b.numpy())

In [ ]:
ds=test_generator
predictions= model.predict(ds)
print("10 Sample predictions:")
for (pred,(a,b)) in zip(predictions,ds):
  
  pred[pred>0.5]=1
  pred[pred<=0.5]=0
  print("predicted: " ,pred, str( pred),  
        "Actual Label: ("+str(b) +")")
  y.append(b)